In [356]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
from datetime import datetime

### Description of the Columns
- 1) year | The year the month occurred
- 2) month | The month for which data was collected
- 3) carrier The airline code
- 4) carrier_name | The airline name
- 5) airport | The airport code
- 6) airport_name | The airport name
- 7) arr_flights | # of flights that arrived at the airport
- 8) arr_del15 | # of flights that arrived >= 15 minutes late
- 9) carrier_ct | # of flights delayed due to the carrier
- 10) weather_ct | # of flights delayed due to weather
- 11) nas_ct | # of flights delayed due to national air system
- 12) security_ct | # of flights delayed due to security
- 13) late_aircraft_ct | # flights delayed because a previous flight using the same aircraft was late
- 14) arr_cancelled | # of canceled arrivals
- 15) arr_diverted | # of scheduled arrivals that were diverted
- 16) arr_delay | Sum of the delay minutes
- 17) carrier_delay | Total minutes of delays due to carriers
- 18) weather_delay | Total minutes of delays due to weather
- 19) nas_delay | Total minutes of delays due to natl. air service
- 20) security_delay | Total minutes of delays due to security
- 21) late_aircraft_delay | Similar to late_aircraft_ct. The total minutes of delay due to a previous flight using the same aircraft arriving late.

In [ ]:
df_original = pd.read_csv('950307216_52018_2911_airline_delay_causes.csv')

In [379]:
df = df_original.copy()

In [380]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240893 entries, 0 to 240892
Data columns (total 22 columns):
year                   240893 non-null int64
 month                 240893 non-null int64
carrier                240893 non-null object
carrier_name           240893 non-null object
airport                240893 non-null object
airport_name           240893 non-null object
arr_flights            240545 non-null float64
arr_del15              240496 non-null float64
carrier_ct             240545 non-null float64
 weather_ct            240545 non-null float64
nas_ct                 240545 non-null float64
security_ct            240545 non-null float64
late_aircraft_ct       240545 non-null float64
arr_cancelled          240545 non-null float64
arr_diverted           240545 non-null float64
 arr_delay             240545 non-null float64
 carrier_delay         240545 non-null float64
weather_delay          240545 non-null float64
nas_delay              240545 non-null float64
secu

,year,month,carrier,carrier_name,airport,airport_name,arr_flights,arr_del15,carrier_ct,weather_ct,...,late_aircraft_ct,arr_cancelled,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,Unnamed: 21
0,2003,6,AA,American Airlines Inc.,ABQ,"Albuquerque, NM: Albuquerque International Sun...",307.0,56.0,14.68,10.79,...,9.96,1.0,1.0,2530.0,510.0,621.0,676.0,25.0,698.0,NaN
1,2003,6,AA,American Airlines Inc.,ANC,"Anchorage, AK: Ted Stevens Anchorage Internati...",90.0,27.0,7.09,2.00,...,7.16,0.0,0.0,1390.0,271.0,83.0,581.0,0.0,455.0,NaN
2,2003,6,AA,American Airlines Inc.,ATL,"Atlanta, GA: Hartsfield-Jackson Atlanta Intern...",752.0,186.0,33.99,27.82,...,17.53,5.0,0.0,8314.0,1367.0,1722.0,3817.0,139.0,1269.0,NaN
3,2003,6,AA,American Airlines Inc.,AUS,"Austin, TX: Austin - Bergstrom International",842.0,174.0,60.24,20.54,...,40.75,9.0,1.0,8344.0,3040.0,1032.0,1835.0,115.0,2322.0,NaN
4,2003,6,AA,American Airlines Inc.,BDL,"Hartford, CT: Bradley International",383.0,55.0,14.90,8.91,...,16.61,0.0,0.0,3137.0,815.0,574.0,555.0,0.0,1193.0,NaN


In [381]:
# Drop the columns we don't need, 'Unnamed: 21, carrier, airport'
df.drop('Unnamed: 21', axis=1, inplace=True)

# Drop the null entries of the column, arr_flight.
df.dropna(subset=['arr_flights'], inplace=True)

# df[df.arr_del15.isna()]
df.dropna(subset=['arr_del15'], inplace=True)

# Rename the columns, carrier->carrier_code, airport->airport_code
df.rename(columns={'carrier':'carrier_code', 'airport':'airport_code'},
                   inplace=True)
# df.drop('airport', axis=1, inplace=True)
# print(df.head())


df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 240496 entries, 0 to 240892
Data columns (total 21 columns):
year                   240496 non-null int64
 month                 240496 non-null int64
carrier_code           240496 non-null object
carrier_name           240496 non-null object
airport_code           240496 non-null object
airport_name           240496 non-null object
arr_flights            240496 non-null float64
arr_del15              240496 non-null float64
carrier_ct             240496 non-null float64
 weather_ct            240496 non-null float64
nas_ct                 240496 non-null float64
security_ct            240496 non-null float64
late_aircraft_ct       240496 non-null float64
arr_cancelled          240496 non-null float64
arr_diverted           240496 non-null float64
 arr_delay             240496 non-null float64
 carrier_delay         240496 non-null float64
weather_delay          240496 non-null float64
nas_delay              240496 non-null float64
secu

In [383]:
# Get rid of the whitespace in the columns
print(df.columns)
col_list = pd.Series(df.columns)
new_col_list = col_list.apply(lambda x: x.strip())
df.columns = new_col_list
df.columns

Index(['year', ' month', 'carrier_code', 'carrier_name', 'airport_code',
       'airport_name', 'arr_flights', 'arr_del15', 'carrier_ct', ' weather_ct',
       'nas_ct', 'security_ct', 'late_aircraft_ct', 'arr_cancelled',
       'arr_diverted', ' arr_delay', ' carrier_delay', 'weather_delay',
       'nas_delay', 'security_delay', 'late_aircraft_delay'],
      dtype='object')


Index(['year', 'month', 'carrier_code', 'carrier_name', 'airport_code',
       'airport_name', 'arr_flights', 'arr_del15', 'carrier_ct', 'weather_ct',
       'nas_ct', 'security_ct', 'late_aircraft_ct', 'arr_cancelled',
       'arr_diverted', 'arr_delay', 'carrier_delay', 'weather_delay',
       'nas_delay', 'security_delay', 'late_aircraft_delay'],
      dtype='object')

In [384]:
# These columns are recoded in minutes. So, the type of the data should be integer.
# arr_delay, carrier_delay, weather_delay, nas_delay, security_delay, late_aircraft_delay

# Change the type of the columns, 'arr_cancelled, arr_diverted', from float64 to int64
df.arr_cancelled = df.arr_cancelled.astype('int64')
df.arr_diverted = df.arr_diverted.astype('int64')
df.arr_flights = df.arr_flights.astype('int64')
df.arr_del15 = df.arr_del15.astype('int64')
df.arr_delay = df.arr_delay.astype('int64')
df.carrier_delay = df.carrier_delay.astype('int64')
df.weather_delay = df.weather_delay.astype('int64')
df.nas_delay = df.nas_delay.astype('int64')
df.security_delay = df.security_delay.astype('int64')
df.late_aircraft_delay = df.late_aircraft_delay.astype('int64')
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 240496 entries, 0 to 240892
Data columns (total 21 columns):
year                   240496 non-null int64
month                  240496 non-null int64
carrier_code           240496 non-null object
carrier_name           240496 non-null object
airport_code           240496 non-null object
airport_name           240496 non-null object
arr_flights            240496 non-null int64
arr_del15              240496 non-null int64
carrier_ct             240496 non-null float64
weather_ct             240496 non-null float64
nas_ct                 240496 non-null float64
security_ct            240496 non-null float64
late_aircraft_ct       240496 non-null float64
arr_cancelled          240496 non-null int64
arr_diverted           240496 non-null int64
arr_delay              240496 non-null int64
carrier_delay          240496 non-null int64
weather_delay          240496 non-null int64
nas_delay              240496 non-null int64
security_delay      

In [385]:
# Combine the two columns, year and month, and then convert the datatype to datetime

date = pd.DataFrame({'month':df.month, 'year':df.year, 'day':[1]*len(df)})
df['date'] = pd.to_datetime(date, format='%Y%m%d')
# df.drop(['year','month'], axis=1, inplace=True)
# cols = df.columns.tolist()
# new_cols = cols[-1:] + cols[:-1]
# df = df[new_cols]
# df.head()


In [386]:
# Split 'airport_name' into the airport and the city
splt_airport = df.airport_name.str.split(':')
splt_airport = pd.Series(splt_airport)

df['city'] = splt_airport.apply(lambda x: x[0].split(',')[0])
df['state'] = splt_airport.apply(lambda x: x[0].split(',')[1])
df['airport'] = splt_airport.apply(lambda x: x[1])
df.drop(['airport_name'], axis=1, inplace=True)
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 240496 entries, 0 to 240892
Data columns (total 24 columns):
year                   240496 non-null int64
month                  240496 non-null int64
carrier_code           240496 non-null object
carrier_name           240496 non-null object
airport_code           240496 non-null object
arr_flights            240496 non-null int64
arr_del15              240496 non-null int64
carrier_ct             240496 non-null float64
weather_ct             240496 non-null float64
nas_ct                 240496 non-null float64
security_ct            240496 non-null float64
late_aircraft_ct       240496 non-null float64
arr_cancelled          240496 non-null int64
arr_diverted           240496 non-null int64
arr_delay              240496 non-null int64
carrier_delay          240496 non-null int64
weather_delay          240496 non-null int64
nas_delay              240496 non-null int64
security_delay         240496 non-null int64
late_aircraft_delay  

In [392]:
cols = df.columns.tolist()
cols

['year',
 'month',
 'carrier_code',
 'carrier_name',
 'airport_code',
 'arr_flights',
 'arr_del15',
 'carrier_ct',
 'weather_ct',
 'nas_ct',
 'security_ct',
 'late_aircraft_ct',
 'arr_cancelled',
 'arr_diverted',
 'arr_delay',
 'carrier_delay',
 'weather_delay',
 'nas_delay',
 'security_delay',
 'late_aircraft_delay',
 'date',
 'city',
 'state',
 'airport']

In [397]:
cols = df.columns.tolist()
cols

['year',
 'month',
 'carrier_code',
 'carrier_name',
 'airport_code',
 'arr_flights',
 'arr_del15',
 'carrier_ct',
 'weather_ct',
 'nas_ct',
 'security_ct',
 'late_aircraft_ct',
 'arr_cancelled',
 'arr_diverted',
 'arr_delay',
 'carrier_delay',
 'weather_delay',
 'nas_delay',
 'security_delay',
 'late_aircraft_delay',
 'date',
 'city',
 'state',
 'airport']

In [400]:
new_cols

['date',
 'year',
 'month',
 'carrier_code',
 'carrier_name',
 'city',
 'state',
 'airport_code',
 'airportarr_flights',
 'arr_del15',
 'carrier_ct',
 'weather_ct',
 'nas_ct',
 'security_ct',
 'late_aircraft_ct',
 'arr_cancelled',
 'arr_diverted',
 'arr_delay',
 'carrier_delay',
 'weather_delay',
 'nas_delay',
 'security_delay',
 'late_aircraft_delay']

In [403]:
cols = df.columns.tolist()
new_cols = []
new_cols = ['date','year','month','carrier_code','carrier_name',
 'city','state','airport_code','airport','arr_flights',
 'arr_del15','carrier_ct','weather_ct','nas_ct','security_ct',
 'late_aircraft_ct','arr_cancelled','arr_diverted','arr_delay',
 'carrier_delay','weather_delay','nas_delay','security_delay',
 'late_aircraft_delay']
# new_cols.append(cols[-4])
# new_cols = cols[0:4] + cols[-3:] + cols[4:-3]
df = df[new_cols]
df.head()
# new_cols

,date,year,month,carrier_code,carrier_name,city,state,airport_code,airport,arr_flights,...,security_ct,late_aircraft_ct,arr_cancelled,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
0,2003-06-01,2003,6,AA,American Airlines Inc.,Albuquerque,NM,ABQ,Albuquerque International Sunport,307,...,1.48,9.96,1,1,2530,510,621,676,25,698
1,2003-06-01,2003,6,AA,American Airlines Inc.,Anchorage,AK,ANC,Ted Stevens Anchorage International,90,...,0.00,7.16,0,0,1390,271,83,581,0,455
2,2003-06-01,2003,6,AA,American Airlines Inc.,Atlanta,GA,ATL,Hartsfield-Jackson Atlanta International,752,...,1.90,17.53,5,0,8314,1367,1722,3817,139,1269
3,2003-06-01,2003,6,AA,American Airlines Inc.,Austin,TX,AUS,Austin - Bergstrom International,842,...,4.69,40.75,9,1,8344,3040,1032,1835,115,2322
4,2003-06-01,2003,6,AA,American Airlines Inc.,Hartford,CT,BDL,Bradley International,383,...,0.00,16.61,0,0,3137,815,574,555,0,1193


In [409]:
df[df.city.str.contains('-')].city.value_counts()

Scranton/Wilkes-Barre    521
Name: city, dtype: int64

In [410]:
df[df.city.str.contains('-')]

,date,year,month,carrier_code,carrier_name,city,state,airport_code,airport,arr_flights,...,security_ct,late_aircraft_ct,arr_cancelled,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
244,2003-06-01,2003,6,DH,Atlantic Coast Airlines,Scranton/Wilkes-Barre,PA,AVP,Wilkes Barre Scranton International,60,...,0.00,1.00,1,0,606,260,0,272,0,74
1491,2003-07-01,2003,7,DH,Atlantic Coast Airlines,Scranton/Wilkes-Barre,PA,AVP,Wilkes Barre Scranton International,62,...,0.00,2.17,4,0,1186,512,168,322,0,184
2740,2003-08-01,2003,8,DH,Atlantic Coast Airlines,Scranton/Wilkes-Barre,PA,AVP,Wilkes Barre Scranton International,62,...,0.00,0.00,4,0,1064,271,90,703,0,0
3987,2003-09-01,2003,9,DH,Atlantic Coast Airlines,Scranton/Wilkes-Barre,PA,AVP,Wilkes Barre Scranton International,60,...,0.00,1.93,1,0,892,339,61,350,0,142
5227,2003-10-01,2003,10,DH,Atlantic Coast Airlines,Scranton/Wilkes-Barre,PA,AVP,Wilkes Barre Scranton International,62,...,0.00,1.00,0,0,581,321,148,90,0,22
6472,2003-11-01,2003,11,DH,Atlantic Coast Airlines,Scranton/Wilkes-Barre,PA,AVP,Wilkes Barre Scranton International,60,...,0.00,3.29,6,1,1461,501,533,90,0,337
7732,2003-12-01,2003,12,DH,Atlantic Coast Airlines,Scranton/Wilkes-Barre,PA,AVP,Wilkes Barre Scranton International,62,...,0.00,6.00,3,0,1172,612,82,87,0,391
9001,2004-01-01,2004,1,DH,Atlantic Coast Airlines,Scranton/Wilkes-Barre,PA,AVP,Wilkes Barre Scranton International,62,...,0.00,4.00,9,1,1889,941,169,265,0,514
9610,2004-01-01,2004,1,OH,Comair Inc.,Scranton/Wilkes-Barre,PA,AVP,Wilkes Barre Scranton International,187,...,0.00,0.90,5,1,2728,1302,876,430,0,120
10393,2004-02-01,2004,2,DH,Atlantic Coast Airlines,Scranton/Wilkes-Barre,PA,AVP,Wilkes Barre Scranton International,58,...,0.00,8.53,4,0,731,206,23,65,0,437


In [404]:
df.city.value_counts()

Washington               4443
New York                 3951
Chicago                  3419
Portland                 3216
Houston                  3117
Los Angeles              2451
Las Vegas                2439
Phoenix                  2392
Columbus                 2388
San Diego                2368
Detroit                  2355
Denver                   2353
Boston                   2304
Kansas City              2302
Dallas/Fort Worth        2298
Atlanta                  2268
San Francisco            2255
Indianapolis             2252
Raleigh/Durham           2249
Austin                   2246
Pittsburgh               2244
New Orleans              2238
Philadelphia             2237
Minneapolis              2210
Orlando                  2189
Seattle                  2188
St. Louis                2159
Newark                   2143
Baltimore                2091
Rochester                2074
                         ... 
Punta Gorda                12
Del Rio                    11
Hoolehua  

In [421]:
df[df.state.str.contains('TT')]

,date,year,month,carrier_code,carrier_name,city,state,airport_code,airport,arr_flights,...,security_ct,late_aircraft_ct,arr_cancelled,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
133157,2010-12-01,2010,12,CO,Continental Air Lines Inc.,Guam,TT,GUM,Guam International,10,...,0.00,0.68,0,0,429,357,0,55,0,17
134524,2011-01-01,2011,1,CO,Continental Air Lines Inc.,Guam,TT,GUM,Guam International,31,...,0.00,7.14,0,0,1014,266,0,116,0,632
135810,2011-02-01,2011,2,CO,Continental Air Lines Inc.,Guam,TT,GUM,Guam International,29,...,0.00,5.41,0,0,585,96,0,239,0,250
135849,2011-02-01,2011,2,CO,Continental Air Lines Inc.,Saipan,TT,SPN,Francisco C. Ada Saipan International,1,...,0.00,0.00,0,0,0,0,0,0,0,0
137101,2011-03-01,2011,3,CO,Continental Air Lines Inc.,Guam,TT,GUM,Guam International,32,...,0.00,4.32,0,0,280,108,0,49,0,123
137140,2011-03-01,2011,3,CO,Continental Air Lines Inc.,Saipan,TT,SPN,Francisco C. Ada Saipan International,1,...,0.00,0.00,0,0,0,0,0,0,0,0
138397,2011-04-01,2011,4,CO,Continental Air Lines Inc.,Guam,TT,GUM,Guam International,31,...,0.00,3.00,0,0,205,0,0,48,0,157
139684,2011-05-01,2011,5,CO,Continental Air Lines Inc.,Guam,TT,GUM,Guam International,31,...,0.00,1.85,0,0,179,56,0,56,0,67
139720,2011-05-01,2011,5,CO,Continental Air Lines Inc.,Saipan,TT,SPN,Francisco C. Ada Saipan International,1,...,0.00,0.00,0,0,0,0,0,0,0,0
140974,2011-06-01,2011,6,CO,Continental Air Lines Inc.,Guam,TT,GUM,Guam International,30,...,0.00,2.00,0,0,181,55,0,14,0,112


In [416]:
df[df.state=='CA']

,date,year,month,carrier_code,carrier_name,city,state,airport_code,airport,arr_flights,...,security_ct,late_aircraft_ct,arr_cancelled,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay


In [417]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 240496 entries, 0 to 240892
Data columns (total 24 columns):
date                   240496 non-null datetime64[ns]
year                   240496 non-null int64
month                  240496 non-null int64
carrier_code           240496 non-null object
carrier_name           240496 non-null object
city                   240496 non-null object
state                  240496 non-null object
airport_code           240496 non-null object
airport                240496 non-null object
arr_flights            240496 non-null int64
arr_del15              240496 non-null int64
carrier_ct             240496 non-null float64
weather_ct             240496 non-null float64
nas_ct                 240496 non-null float64
security_ct            240496 non-null float64
late_aircraft_ct       240496 non-null float64
arr_cancelled          240496 non-null int64
arr_diverted           240496 non-null int64
arr_delay              240496 non-null int64
carrier_d

In [422]:
df.arr_delay

<class 'pandas.core.frame.DataFrame'>
Int64Index: 240496 entries, 0 to 240892
Data columns (total 24 columns):
date                   240496 non-null datetime64[ns]
year                   240496 non-null int64
month                  240496 non-null int64
carrier_code           240496 non-null object
carrier_name           240496 non-null object
city                   240496 non-null object
state                  240496 non-null object
airport_code           240496 non-null object
airport                240496 non-null object
arr_flights            240496 non-null int64
arr_del15              240496 non-null int64
carrier_ct             240496 non-null float64
weather_ct             240496 non-null float64
nas_ct                 240496 non-null float64
security_ct            240496 non-null float64
late_aircraft_ct       240496 non-null float64
arr_cancelled          240496 non-null int64
arr_diverted           240496 non-null int64
arr_delay              240496 non-null int64
carrier_d

In [411]:
df.to_csv("clean_flights_delay.csv", index=False)

In [59]:
airline_codes[airline_codes.Airline.str.contains('Republic')]

,carrier,ICAO,Airline,Call sign,Country/Region,Comments
54,RW,RPA,Republic Airlines,BRICKYARD,United States,NaN


In [402]:
df

,year,month,carrier_code,carrier_name,airport_code,arr_flights,arr_del15,carrier_ct,weather_ct,nas_ct,...,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,date,city,state,airport
0,2003,6,AA,American Airlines Inc.,ABQ,307,56,14.68,10.79,19.09,...,2530,510,621,676,25,698,2003-06-01,Albuquerque,NM,Albuquerque International Sunport
1,2003,6,AA,American Airlines Inc.,ANC,90,27,7.09,2.00,10.75,...,1390,271,83,581,0,455,2003-06-01,Anchorage,AK,Ted Stevens Anchorage International
2,2003,6,AA,American Airlines Inc.,ATL,752,186,33.99,27.82,104.76,...,8314,1367,1722,3817,139,1269,2003-06-01,Atlanta,GA,Hartsfield-Jackson Atlanta International
3,2003,6,AA,American Airlines Inc.,AUS,842,174,60.24,20.54,47.78,...,8344,3040,1032,1835,115,2322,2003-06-01,Austin,TX,Austin - Bergstrom International
4,2003,6,AA,American Airlines Inc.,BDL,383,55,14.90,8.91,14.57,...,3137,815,574,555,0,1193,2003-06-01,Hartford,CT,Bradley International
5,2003,6,AA,American Airlines Inc.,BHM,89,12,2.79,2.19,2.20,...,673,94,90,50,35,404,2003-06-01,Birmingham,AL,Birmingham-Shuttlesworth International
6,2003,6,AA,American Airlines Inc.,BNA,445,82,25.44,11.98,17.90,...,4663,1217,913,800,1,1732,2003-06-01,Nashville,TN,Nashville International
7,2003,6,AA,American Airlines Inc.,BOS,1266,225,69.43,23.66,83.93,...,12139,4201,1783,3067,45,3043,2003-06-01,Boston,MA,Logan International
8,2003,6,AA,American Airlines Inc.,BUR,119,27,7.49,4.65,7.81,...,1187,326,191,285,0,385,2003-06-01,Burbank,CA,Bob Hope
9,2003,6,AA,American Airlines Inc.,BWI,593,101,17.56,20.49,38.26,...,5698,1058,1332,1708,0,1600,2003-06-01,Baltimore,MD,Baltimore/Washington International Thurgood M...


In [44]:
iata = df[['carrier','carrier_name']]

In [53]:
dd = pd.merge(iata, airline_codes, on='carrier', how='left')

In [56]:
dd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 394488 entries, 0 to 394487
Data columns (total 7 columns):
carrier           394488 non-null object
carrier_name      394488 non-null object
ICAO              361190 non-null object
Airline           368107 non-null object
Call sign         368107 non-null object
Country/Region    368107 non-null object
Comments          172058 non-null object
dtypes: object(7)
memory usage: 24.1+ MB


In [60]:
dd

,carrier,carrier_name,ICAO,Airline,Call sign,Country/Region,Comments
0,AA,American Airlines Inc.,AAL,American Airlines,AMERICAN,United States,NaN
1,AA,American Airlines Inc.,AAL,American Airlines,AMERICAN,United States,NaN
2,AA,American Airlines Inc.,AAL,American Airlines,AMERICAN,United States,NaN
3,AA,American Airlines Inc.,AAL,American Airlines,AMERICAN,United States,NaN
4,AA,American Airlines Inc.,AAL,American Airlines,AMERICAN,United States,NaN
5,AA,American Airlines Inc.,AAL,American Airlines,AMERICAN,United States,NaN
6,AA,American Airlines Inc.,AAL,American Airlines,AMERICAN,United States,NaN
7,AA,American Airlines Inc.,AAL,American Airlines,AMERICAN,United States,NaN
8,AA,American Airlines Inc.,AAL,American Airlines,AMERICAN,United States,NaN
9,AA,American Airlines Inc.,AAL,American Airlines,AMERICAN,United States,NaN


In [69]:
dd[dd.Airline.isna()].groupby('carrier').count()

,carrier_name,ICAO,Airline,Call sign,Country/Region,Comments
carrier,,,,,,
RU,4136,0,0,0,0,0
UA,14466,0,0,0,0,0
XE,7375,0,0,0,0,0
YX,404,0,0,0,0,0


In [76]:
dd[dd.carrier == 'XE']

,carrier,carrier_name,ICAO,Airline,Call sign,Country/Region,Comments
94779,XE,ExpressJet Airlines Inc.,NaN,NaN,NaN,NaN,NaN
94780,XE,ExpressJet Airlines Inc.,NaN,NaN,NaN,NaN,NaN
94781,XE,ExpressJet Airlines Inc.,NaN,NaN,NaN,NaN,NaN
94782,XE,ExpressJet Airlines Inc.,NaN,NaN,NaN,NaN,NaN
94783,XE,ExpressJet Airlines Inc.,NaN,NaN,NaN,NaN,NaN
94784,XE,ExpressJet Airlines Inc.,NaN,NaN,NaN,NaN,NaN
94785,XE,ExpressJet Airlines Inc.,NaN,NaN,NaN,NaN,NaN
94786,XE,ExpressJet Airlines Inc.,NaN,NaN,NaN,NaN,NaN
94787,XE,ExpressJet Airlines Inc.,NaN,NaN,NaN,NaN,NaN
94788,XE,ExpressJet Airlines Inc.,NaN,NaN,NaN,NaN,NaN


In [74]:
airline_codes[airline_codes.Airline.str.contains('United')]

,carrier,ICAO,Airline,Call sign,Country/Region,Comments
